In [1]:
# Read config
import configparser

config = configparser.ConfigParser()
config.read("config.secret")


['config.secret']

# ELOQUA

In [2]:
import requests

# Credentials saved in 1password under "Eloqua API"
AUTH = (config["Eloqua"]["username"], config["Eloqua"]["password"])
ELOQUA_URL = "https://secure.p01.eloqua.com/API/REST/2.0"


def get_eloqua(path):
    api_url = ELOQUA_URL + path
    response = requests.get(api_url, auth=AUTH)
    return response.json()


In [3]:
# Get schema for all lead records
get_eloqua("/assets/customObject/462")


{'type': 'CustomObject',
 'id': '462',
 'createdAt': '1430937353',
 'createdBy': '108',
 'depth': 'complete',
 'description': 'Inbound only Sync from DCRM (SFA)',
 'folderId': '30740',
 'name': 'DCRM - ALL Leads',
 'updatedAt': '1614100345',
 'updatedBy': '108',
 'deleteLinkedCustomObjectData': 'false',
 'displayNameFieldId': '6782',
 'fields': [{'type': 'CustomObjectField',
   'id': '6772',
   'depth': 'complete',
   'name': 'Email Address',
   'dataType': 'text',
   'displayType': 'text',
   'internalName': 'Email_Address1'},
  {'type': 'CustomObjectField',
   'id': '15579',
   'depth': 'complete',
   'name': 'DCRM Lead ID',
   'dataType': 'text',
   'displayType': 'text',
   'internalName': 'DCRM___Lead_ID1'},
  {'type': 'CustomObjectField',
   'id': '6783',
   'depth': 'complete',
   'name': 'LMP Lead ID',
   'dataType': 'text',
   'displayType': 'text',
   'internalName': 'Lead_Portal_ID1'},
  {'type': 'CustomObjectField',
   'id': '6773',
   'depth': 'complete',
   'name': 'DCRM 

Fields of interest:

```py
  {'type': 'CustomObjectField',
   'id': '6772',
   'depth': 'complete',
   'name': 'Email Address',
   'dataType': 'text',
   'displayType': 'text',
   'internalName': 'Email_Address1'}

  {'type': 'CustomObjectField',
   'id': '6780',
   'depth': 'complete',
   'name': 'Status',
   'dataType': 'text',
   'displayType': 'singleSelect',
   'internalName': 'Status1',
   'optionListId': '24097'}

  {'type': 'CustomObjectField',
   'id': '6775',
   'depth': 'complete',
   'name': 'Sales Accepted',
   'dataType': 'text',
   'displayType': 'singleSelect',
   'internalName': 'Sales_Accepted1',
   'optionListId': '24638'}

  {'type': 'CustomObjectField',
   'id': '6777',
   'depth': 'complete',
   'name': 'Sales Qualified',
   'dataType': 'text',
   'displayType': 'singleSelect',
   'internalName': 'Sales_Qualified1',
   'optionListId': '24638'}

  {'type': 'CustomObjectField',
   'id': '6825',
   'depth': 'complete',
   'name': 'Tele Accepted',
   'dataType': 'text',
   'displayType': 'singleSelect',
   'internalName': 'Tele_Accepted1',
   'optionListId': '24637'}

  {'type': 'CustomObjectField',
   'id': '6828',
   'depth': 'complete',
   'name': 'Tele Qualified',
   'dataType': 'text',
   'displayType': 'singleSelect',
   'internalName': 'Tele_Qualified1',
   'optionListId': '24637'}

  {'type': 'CustomObjectField',
   'id': '27198',
   'depth': 'complete',
   'name': 'DCRM Lead Stage',
   'dataType': 'text',
   'displayType': 'singleSelect',
   'internalName': 'DCRM_Lead_Stage1',
   'optionListId': '24640'}

  {'type': 'CustomObjectField',
   'id': '6822',
   'depth': 'complete',
   'name': 'LMP Status',
   'dataType': 'text',
   'displayType': 'text',
   'internalName': 'LMP_Status1'}

  {'type': 'CustomObjectField',
   'id': '6821',
   'depth': 'complete',
   'name': 'LMP Opportunity Value',
   'dataType': 'numeric',
   'displayType': 'text',
   'internalName': 'LMP_Opportunity_Value1'}
```

# Adobe Analytics

In [4]:
import requests
import jwt
import datetime

# Credentials saved on 1password under "Adobe Analytics API"

JWT_URL = "https://ims-na1.adobelogin.com/ims/exchange/jwt/"

# From Adobe Developer Console + generated expiry date
JWT_PAYLOAD = {
    "iss": config["Adobe Analytics"]["jwt_iss"],
    "sub": config["Adobe Analytics"]["jwt_sub"],
    "https://ims-na1.adobelogin.com/s/ent_analytics_bulk_ingest_sdk": True,
    "aud": config["Adobe Analytics"]["jwt_aud"],
    "exp": int((datetime.datetime.now() + datetime.timedelta(days=1)).strftime("%s")),
}
jwt_payload_encrypted = jwt.encode(
    JWT_PAYLOAD, config["Adobe Analytics"]["private_key"], algorithm="RS256"
)

access_token_request_payload = {
    "client_id": config["Adobe Analytics"]["client_id"],
    "client_secret": config["Adobe Analytics"]["client_secret"],
    "jwt_token": jwt_payload_encrypted,
}

response = requests.post(url=JWT_URL, data=access_token_request_payload)
ACCESS_TOKEN = response.json()["access_token"]

# Get global company ID
DISCOVERY_URL = "https://analytics.adobe.io/discovery/me"
DISCOVERY_HEADER = {
    "Accept": "application/json",
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "x-api-key": config["Adobe Analytics"]["client_id"],
}
response = requests.get(url=DISCOVERY_URL, headers=DISCOVERY_HEADER)
GLOBAL_COMPANY_ID = response.json()["imsOrgs"][0]["companies"][0]["globalCompanyId"]
# globalCompanyId: rockwe1

response.json()


{'imsUserId': 'E4231E6061F8463E0A495C3E@techacct.adobe.com',
 'imsOrgs': [{'imsOrgId': '355141B554736E1B0A4C98A7@AdobeOrg',
   'companies': [{'globalCompanyId': 'rockwe1',
     'companyName': 'rockwellautomation',
     'apiRateLimitPolicy': 'aa_api_tier10_tp',
     'dpc': 'pnw'}]}]}

In [5]:
# This is the complete HEADER
HEADER = {
    "Accept": "application/json",
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "x-api-key": config["Adobe Analytics"]["client_id"],
    "x-proxy-global-company-id": GLOBAL_COMPANY_ID,
}
BASE_URL = f"https://analytics.adobe.io/api/{GLOBAL_COMPANY_ID}"

# Get report suite ID
FIND_RSID_URL = BASE_URL + "/collections/suites"
response = requests.get(url=FIND_RSID_URL, headers=HEADER)
RSID = response.json()["content"][0]["rsid"]

# Use BASE_URL + endpoint + RSID_ADDON for requests
RSID_ADDON = f"?rsid={RSID}"
response.json()["content"]


[{'collectionItemType': 'reportsuite',
  'id': 'rokrockwell-locator-dev',
  'rsid': 'rokrockwell-locator-dev',
  'name': 'Rockwell Corp Locator (Dev)'},
 {'collectionItemType': 'reportsuite',
  'id': 'rokrockwell-partner-dev',
  'rsid': 'rokrockwell-partner-dev',
  'name': 'Rockwell Corp Partner (Dev)'},
 {'collectionItemType': 'reportsuite',
  'id': 'rokrockwellassetsinsights',
  'rsid': 'rokrockwellassetsinsights',
  'name': 'AEM Assets Insights (prod)'},
 {'collectionItemType': 'reportsuite',
  'id': 'rokrockwellassetsinsightsdev',
  'rsid': 'rokrockwellassetsinsightsdev',
  'name': 'AEM Assets Insights (Dev)'},
 {'collectionItemType': 'reportsuite',
  'id': 'rokrockwellcorpdev',
  'rsid': 'rokrockwellcorpdev',
  'name': 'Rockwell Corp (Dev)'},
 {'collectionItemType': 'reportsuite',
  'id': 'rokrockwellcorpprod',
  'rsid': 'rokrockwellcorpprod',
  'name': 'Rockwell Corp (Prod)'},
 {'collectionItemType': 'reportsuite',
  'id': 'rokrockwelllocatorprod',
  'rsid': 'rokrockwelllocatorpr

In [6]:
import pandas as pd

url = BASE_URL + "/metrics" + RSID_ADDON
response = requests.get(url=url, headers=HEADER)
df = pd.DataFrame(response.json())
df


,id,title,name,type,category,support,allocation,precision,calculated,segmentable,supportsDataGovernance,polarity,description,extraTitleInfo,helpLink
0,metrics/averagepagedepth,Average Page Depth,Average Page Depth,int,Traffic,[oberon],False,0.0,False,False,False,positive,NaN,NaN,NaN
1,metrics/averagetimespentonpage,Average Time Spent on Page (seconds),Average Time Spent on Page (seconds),int,Traffic,[],False,0.0,False,False,False,positive,NaN,NaN,NaN
2,metrics/averagetimespentonsite,Average Time Spent on Site (seconds),Average Time Spent on Site (seconds),int,Traffic,[oberon],False,0.0,False,False,False,positive,NaN,NaN,NaN
3,metrics/averagevisitdepth,Average Visit Depth,Average Visit Depth,int,Traffic,[],False,0.0,False,False,False,positive,NaN,NaN,NaN
4,metrics/bouncerate,Bounce Rate,Bounce Rate,percent,Traffic,[oberon],False,2.0,True,False,False,negative,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,metrics/visitorsmonthly,Monthly Unique Visitors,Monthly Unique Visitors,int,Traffic,[],False,0.0,False,False,False,positive,NaN,NaN,NaN
201,metrics/visitorsquarterly,Quarterly Unique Visitors,Quarterly Unique Visitors,int,Traffic,[],False,0.0,False,False,False,positive,NaN,NaN,NaN
202,metrics/visitorsweekly,Weekly Unique Visitors,Weekly Unique Visitors,int,Traffic,[],False,0.0,False,False,False,positive,NaN,NaN,NaN
203,metrics/visitorsyearly,Yearly Unique Visitors,Yearly Unique Visitors,int,Traffic,[],False,0.0,False,False,False,positive,NaN,NaN,NaN


In [7]:
POST_REPORT_URL = BASE_URL + "reports" + RSID_ADDON

START_DATE = "2022-01-01"
END_DATE = "2022-02-01"
MIDNIGHT = "T00:00:00.000"
DATE_RANGE = START_DATE + MIDNIGHT + "/" + END_DATE + MIDNIGHT

METS = ["metrics/visits", "metrics/orders", "metrics/revenue"]
METS_OBJ = [{"id": x} for x in METS]

DIM = "variables/lasttouchchannel"

REPORT_BODY = {
    "rsid": RSID,
    "globalFilters": [{"type": "dateRange", "dateRange": DATE_RANGE}],
    "metricContainer": {
        "metrics": METS_OBJ,
    },
    "dimension": DIM,
    "settings": {
        "dimensionSort": "asc",
        "limit": 5,
    },
}

response = requests.post(url=POST_REPORT_URL, headers=HEADER, json=REPORT_BODY)
# df = pd.DataFrame(response.json())
response.json()


{'error_code': '403025', 'message': 'Profile is not valid'}